## 0. Imports

In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np

from hydra.core.global_hydra import GlobalHydra
from hydra import compose, initialize

import sys

sys.path.append("../")

from di_nn.datasets import DistributedSSLDataset
from di_nn.di_ssl_net import DISSLNET
from di_nn.trainer import DISSLNETLightniningModule

GlobalHydra.instance().clear()
initialize(config_path="../config")
config = compose("config")

MODEL_CHECKPOINT_PATH = "./pretrained_weights.ckpt"
DEMO_DATASET_PATH = "./testing_samples"

## 1. Load model and dataset

In [2]:
dataset = DistributedSSLDataset(DEMO_DATASET_PATH)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
    drop_last=False,
    num_workers=1
)

dataloader = iter(dataloader)

model = DISSLNETLightniningModule(config)
model.eval()
torch.set_grad_enabled(False)
checkpoint = torch.load(MODEL_CHECKPOINT_PATH, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint["state_dict"])


<All keys matched successfully>

## 2. Evaluate dataset samples on model

In [ ]:
while True:
    try:
        batch = next(dataloader)
    except StopIteration:
        break
    model_output = model(batch[0])[0].numpy()

    true_coords = batch[1]["source_coordinates"][0].numpy()
    
    print("True vs estimated coordinates:", true_coords, model_output)
    #print("Predicted coordinates:", model_output)
    print("Error (meters):", np.linalg.norm(true_coords - model_output))
    print("\n")

True vs estimated coordinates: [2.7 1.2] [2.7098007 1.1963917]
Error (meters): 0.010443819


True vs estimated coordinates: [4.3 1.2] [4.399105  1.1978729]
Error (meters): 0.09912771


True vs estimated coordinates: [7.5 1.2] [7.4723206 1.2005477]
Error (meters): 0.027684862


True vs estimated coordinates: [7.5 1.2] [6.955812  1.2071037]
Error (meters): 0.5442344


